In [ ]:
from sklearn.preprocessing import LabelEncoder
from glob import glob
from tqdm import tqdm
import pandas as pd

In [ ]:
CONST = flow_package.Const()

TRAIN_DIR = "../../../data_cicids2017/1_sampling"

files_path = glob(f"{TRAIN_DIR}/*.csv")

df = pd.DataFrame()
for file_path in tqdm(files_path):
    df_tmp = pd.read_csv(file_path)
    df_tmp = df_tmp.drop(columns=["Label", "Attempted Category"])
    df_tmp = df_tmp.dropna()
    df_tmp = df_tmp.drop_duplicates()
    df = pd.concat([df, df_tmp], axis=0)

# Label
le = LabelEncoder()
df["Label"] = le.fit_transform(df["Label"])

df["label"] = df["Label"].apply(lambda x: 0 if x == 0 else 1)